In [ ]:
!git clone https://github.com/Paritosh97/SGNify.git
%cd SGNify

In [ ]:
!echo -e "\nDownloading SGNify..."
!wget --post-data "username=paritosh.sharmas@gmail.com&password=12345678" 'https://download.is.tue.mpg.de/download.php?domain=sgnify&resume=1&sfile=data.zip' -O 'data.zip' --no-check-certificate --continue
!unzip data.zip
!rm data.zip
!mkdir data/coco17
!mv data/vitpose-h-multi-coco.pth data/coco17/vitpose-huge.pth

In [ ]:
!GIT_LFS_SKIP_SMUDGE=1 git submodule update --init --recursive
!git submodule update --init --recursive

In [ ]:
%cd spectre
!pip3 install gdown
!echo -e "\nDownload pretrained SPECTRE model..."
!gdown --id 1vmWX6QmXGPnXTXWFgj67oHzOoOmxBh6B
!mkdir -p pretrained/
!mv spectre_model.tar pretrained/
import os

file_path = "data/template.mtl"

# Check if the file exists
if not os.path.isfile(file_path):
    # File does not exist, create it and write the content
    with open(file_path, "w") as file:
        file.write("newmtl FaceTexture\nmap_Kd mean_texture.jpg")
    print(f"File {file_path} created with the specified content.")
else:
    # File exists, overwrite it with the new content
    with open(file_path, "w") as file:
        file.write("newmtl FaceTexture\nmap_Kd mean_texture.jpg")
    print(f"File {file_path} already existed and has been overwritten with the specified content.")
# Create the target directory if it doesn't exist
!mkdir -p ./external/face_detection/ibug/face_detection/retina_face/weights/
# Download the files using wget
!wget "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/mobilenet0.25_Final.pth" -O ./external/face_detection/ibug/face_detection/retina_face/weights/mobilenet0.25_Final.pth
!wget "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/Resnet50_Final.pth" -O ./external/face_detection/ibug/face_detection/retina_face/weights/Resnet50_Final.pth
%cd ..
!cp -r data/FLAME2020 spectre/data/FLAME2020

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python sgnify.py --image_dir_path ./data/demo/frame_low --output_folder ./data/demo/output/frame_low   --output_name output_frame_low

In [ ]:
# test on 1rjp
!wget -O ./data/demo/video.mp4 "https://nextcloud.lisn.upsaclay.fr/index.php/s/PZnG2sSFbRtrLEz/download/1R-JP_2.mp"
!python sgnify.py --video_path ./data/demo/video.mp4 --output_folder ./data/demo/output/ --output_name output_1rjp